# IBM WatsonX AI & Exa Web Search Integration Notebook

This notebook integrates IBM WatsonX AI with Exa's web search. It accepts a question, decides if a web search is needed, executes the search, and then returns an answer with sources.

Make sure your `.env` file has the following variables:

- IBM_WATSONX_API_KEY
- IBM_WATSONX_URL
- IBM_WATSONX_PROJECT_ID
- EXA_API_KEY

In [ ]:
# Install dependencies if not already installed
%pip install python-dotenv ibm-watsonx-ai exa-py

import os
import json
import logging
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from ibm_watsonx_ai import APIClient, Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

## 1. IBM WatsonX AI Client Setup

In [9]:
class WatsonXClient:
    """Wrapper for IBM WatsonX AI client with project ID handling."""

    def __init__(self):
        # Load environment variables
        load_dotenv()

        self.api_key = os.getenv('IBM_WATSONX_API_KEY')
        self.url = os.getenv('IBM_WATSONX_URL')
        self.project_id = os.getenv('IBM_WATSONX_PROJECT_ID')

        if not all([self.api_key, self.url, self.project_id]):
            raise ValueError(
                "Missing required environment variables. "
                "Please ensure IBM_WATSONX_API_KEY, IBM_WATSONX_URL, and IBM_WATSONX_PROJECT_ID "
                "are set in .env file"
            )

        # Create credentials object
        self.credentials = Credentials(
            url=self.url,
            api_key=self.api_key
        )

        # Create API client
        self.client = APIClient(credentials=self.credentials, project_id=self.project_id)

def get_client() -> WatsonXClient:
    return WatsonXClient()


## 2. Exa Web Search Client

In [10]:
from exa_py import Exa  # Make sure exa-py is installed

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ExaSearchClient:
    """Client for performing web searches using Exa."""

    def __init__(self, api_key: str):
        self.client = Exa(api_key=api_key)

    def search(self, query: str, num_results: int = 3) -> List[Dict[str, Any]]:
        try:
            logger.info(f"Searching Exa for: {query}")
            response = self.client.search_and_contents(
                query,
                type="auto",
                num_results=5,
                text=True,
            )
            
            logger.debug(f"Raw Exa response: {response}")
            
            formatted_results = []
            results = []
            if isinstance(response, dict):
                results = response.get('data', {}).get('results', [])
            else:
                # If response is an object, try to access results directly
                results = getattr(response, 'results', [])

            for result in results:
                if isinstance(result, dict):
                    text = result.get('text', '')
                    title = result.get('title', 'No title')
                    url = result.get('url', '')
                    score = result.get('score', 0.0)
                    published_date = result.get('publishedDate', '')
                else:
                    text = getattr(result, 'text', '')
                    title = getattr(result, 'title', 'No title')
                    url = getattr(result, 'url', '')
                    score = getattr(result, 'score', 0.0)
                    published_date = getattr(result, 'publishedDate', '')
                
                if text:
                    formatted_results.append({
                        "title": title,
                        "text": text[:500] + "..." if len(text) > 500 else text,
                        "url": url,
                        "score": score,
                        "published_date": published_date
                    })

            logger.info(f"\nFound {len(formatted_results)} results using Exa\n")
            if not formatted_results:
                logger.warning("No results found in the response")

            return formatted_results
        except Exception as e:
            logger.error(f"Error searching Exa: {str(e)}", exc_info=True)
            return []


## 3. Tools for WatsonX AI Tool Calling

In [11]:
def create_web_search_tool() -> Dict[str, Any]:
    return {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for current information. Always cite sources.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }

class ToolExecutor:
    def __init__(self, exa_client: ExaSearchClient):
        self.exa_client = exa_client

    def execute_tool(self, tool_call: Dict[str, Any]) -> str:
        if tool_call["type"] != "function":
            return "Unsupported tool type"
        
        function_call = tool_call["function"]
        if function_call["name"] == "web_search":
            try:
                query = json.loads(function_call["arguments"])["query"]
                results = self.exa_client.search(query)

                formatted_results = []
                for result in results:
                    formatted_results.append(
                        f"Source: {result['title']}\n"\
                        f"URL: {result['url']}\n"\
                        f"Content: {result['text']}\n"
                    )

                return "\n\n".join(formatted_results)
            except Exception as e:
                return f"Error executing web search: {str(e)}"

        return "Unknown tool"


## 4. WatsonX AI Model (Granite) with Tool Calling

In [12]:
class WatsonXModel:
    def __init__(self, exa_api_key: Optional[str] = None):
        self.client_wrapper = get_client()

        # Initialize Exa client and tool executor
        self.exa_client = ExaSearchClient(exa_api_key) if exa_api_key else None
        self.tool_executor = ToolExecutor(self.exa_client) if self.exa_client else None

        # Initialize the model (using Mistral for tool calling, as example)
        self.model = ModelInference(
            model_id="ibm/granite-3-8b-instruct",
            credentials=self.client_wrapper.credentials,
            project_id=self.client_wrapper.project_id
        )

    def ask(self, question: str) -> str:
        if not self.tool_executor:
            return "Exa API key not provided"

        try:
            messages = [{"role": "user", "content": question}]
            tools = [create_web_search_tool()]

            # First attempt to get a response (model can request a tool)
            response = self.model.chat(messages=messages, tools=tools)

            if "choices" in response and response["choices"]:
                choice = response["choices"][0]
                if "message" in choice and "tool_calls" in choice["message"]:
                    tool_calls = choice["message"]["tool_calls"]

                    tool_results = []
                    for tool_call in tool_calls:
                        result = self.tool_executor.execute_tool(tool_call)
                        tool_results.append(result)

                    # Add these tool results to the conversation
                    messages.extend([
                        {
                            "role": "assistant",
                            "content": None,
                            "tool_calls": tool_calls
                        },
                        {
                            "role": "tool",
                            "content": "\n\n".join(tool_results),
                            "tool_call_id": tool_calls[0]["id"]
                        }
                    ])

                    final_response = self.model.chat(messages=messages)
                    if "choices" in final_response and final_response["choices"]:
                        return final_response["choices"][0]["message"]["content"].strip()

            return "No response generated"
        except Exception as e:
            logger.error(f"Error: {str(e)}")
            return f"Error: {str(e)}"

## 5. Example Usage

In [13]:
def run_example():
    # Make sure you have a .env file or environment variables set:
    # IBM_WATSONX_API_KEY, IBM_WATSONX_URL, IBM_WATSONX_PROJECT_ID, EXA_API_KEY
    load_dotenv()
    exa_api_key = os.getenv('EXA_API_KEY')
    if not exa_api_key:
        print("Error: EXA_API_KEY not found in environment variables")
        return

    model = WatsonXModel(exa_api_key=exa_api_key)

    # Ask something that might need a web search
    question = "Tell me the recent tech news of 2025?"
    
    print("\nQuestion:", question)
    print("\nProcessing...\n")

    answer = model.ask(question)
    print("\nAnswer:", answer)


## 6. Run the Example

In [14]:
run_example()

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-01-24&project_id=d7c4caa6-2cf1-4af7-9e69-b1baca2c278f&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'



Question: Tell me the recent tech news of 2025?

Processing...



INFO:httpx:HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2025-01-24 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished chat for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2025-01-24'
INFO:__main__:Searching Exa for: recent tech news 2025
INFO:__main__:
Found 5 results using Exa

INFO:httpx:HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2025-01-24 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished chat for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2025-01-24'



Answer: In 2025, several significant developments in technology were predicted and observed:

1. AI Integration: Artificial Intelligence is expected to have a substantial impact on workforce dynamics, with the potential introduction of AI agents capable of commerce, collaboration, and creativity without human intervention. (Source: CoinDesk)

2. Neuralink: Elon Musk's Neuralink projected more volunteers would get brain implants, but a full product release doesn't seem imminent. The brain-machine interface technology's growth and advancements promise to modify human-computer interaction significantly. (Source: MIT Technology Review)

3. Smart Glasses: By 2025, smart glasses are forecasted to become cool and fashionable, merging seamlessly into everyday life. Companies like Meta, Ray-Ban, Snap, and Google are making significant strides in this area. (Source: Technology Review)

4. AI in Software Development: Mark Zuckerberg and other tech leaders predicted that AI would replace mid-leve